In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [75]:
'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4
'''

'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_16164/COMPINT'
nx=128
ny=192
ncomp=4
'''


pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4


print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT


In [76]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 64, 96, 4) , mean = 147.09867350260416 , std = 30.06938300771007 , Max = 226.0 , Min = 46.0 , type=float64
t_C_0 shape: (1, 64, 96) , mean = 100.33577473958333 , std = 13.21556241556425 , Max = 169.0 , Min = 46.0 , type=float64
t_C_1 shape: (1, 64, 96) , mean = 160.08935546875 , std = 11.139810073102135 , Max = 195.0 , Min = 118.0 , type=float64
t_C_2 shape: (1, 64, 96) , mean = 166.34326171875 , std = 13.898822109368025 , Max = 226.0 , Min = 113.0 , type=float64
t_C_3 shape: (1, 64, 96) , mean = 161.62630208333334 , std = 13.704553146688596 , Max = 208.0 , Min = 93.0 , type=float64


In [77]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (64, 96) , mean = 33.626302083333336 , std = 13.704553146688598 , Max = 80 , Min = -35 , type=int8


In [78]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen6144
size16156
size24471
ratio1= 1.001953125
ratio2= 0.727701822917


In [79]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 6144
mse = 0.0


In [80]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 64, 96, 4) , mean = 147.64937337239584 , std = 29.945285425860344 , Max = 216.0 , Min = 43.0 , type=float64
 channel0 shape: (64, 96) , mean = -28.7119140625 , std = 10.743564024612507 , Max = 34 , Min = -85 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4264 Bytes
ratio1= 1.001953125
ratio2= 0.694010416667
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 33.1201171875 , std = 9.584341151000787 , Max = 88 , Min = -8 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4187 Bytes
ratio1= 1.001953125
ratio2= 0.681477864583
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = 39.416341145833336 , std = 10.798830769476211 , Max = 81 , Min = -40 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4292 Bytes
ratio1= 1.001953125
ratio2= 0.698567708333
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = 34.77294921875 , std = 11.101862234695153 , Max = 76 , Min = -30 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
  

dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = 40.112467447916664 , std = 20.54750785223703 , Max = 125 , Min = -40 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 5020 Bytes
ratio1= 1.001953125
ratio2= 0.817057291667
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = 35.911946614583336 , std = 18.810143830503616 , Max = 105 , Min = -30 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4938 Bytes
ratio1= 1.001953125
ratio2= 0.8037109375
dlen= 6144
mse = 0.0



9
  t_C shape: (1, 64, 96, 4) , mean = 148.92936197916666 , std = 31.72881958158617 , Max = 230.0 , Min = 31.0 , type=float64
 channel0 shape: (64, 96) , mean = -29.738606770833332 , std = 11.461341005407954 , Max = 40 , Min = -97 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4172 Bytes
ratio1= 1.001953125
ratio2= 0.679036458333
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 35.093587239583336 , std = 10.63886340760471 , Max = 102 , Min = -18 , type=int8
   ol

dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 28.638671875 , std = 10.843673837707588 , Max = 61 , Min = -10 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4332 Bytes
ratio1= 1.001953125
ratio2= 0.705078125
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = 33.468587239583336 , std = 12.775021004913572 , Max = 90 , Min = -10 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4487 Bytes
ratio1= 1.001953125
ratio2= 0.730305989583
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = 29.461751302083332 , std = 12.490237148119899 , Max = 69 , Min = -13 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4478 Bytes
ratio1= 1.001953125
ratio2= 0.728841145833
dlen= 6144
mse = 0.0



17
  t_C shape: (1, 64, 96, 4) , mean = 141.43070475260416 , std = 24.440228886253056 , Max = 220.0 , Min = 33.0 , type=float64
 channel0 shape: (64, 96) , mean = -19.217122395833332 , std = 14.14607318343366 , Max = 40 , Min = -95 , type=int8
   olen: 614

   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4564 Bytes
ratio1= 1.001953125
ratio2= 0.742838541667
dlen= 6144
mse = 0.0



24
  t_C shape: (1, 64, 96, 4) , mean = 146.69287109375 , std = 29.49212975367941 , Max = 234.0 , Min = 33.0 , type=float64
 channel0 shape: (64, 96) , mean = -25.930989583333332 , std = 13.628563775512989 , Max = 51 , Min = -95 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4506 Bytes
ratio1= 1.001953125
ratio2= 0.7333984375
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 31.265625 , std = 12.269837387072753 , Max = 74 , Min = -10 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4394 Bytes
ratio1= 1.001953125
ratio2= 0.715169270833
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = 36.728841145833336 , std = 15.837997442226518 , Max = 103 , Min = -5 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4690 Bytes
ratio1= 1.001953125
ratio2= 0.763346354167
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , 

In [81]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.001953125
   ratio_2= 0.69482421875
 
2
   ratio_1= 1.001953125
   ratio_2= 0.589111328125
 
3
   ratio_1= 1.001953125
   ratio_2= 0.6958414713541666
 
4
   ratio_1= 1.001953125
   ratio_2= 0.6709391276041666
 
5
   ratio_1= 1.001953125
   ratio_2= 0.7783610026041666
 
6
   ratio_1= 1.001953125
   ratio_2= 0.7744140625
 
7
   ratio_1= 1.001953125
   ratio_2= 0.7138671875
 
8
   ratio_1= 1.001953125
   ratio_2= 0.7871500651041666
 
9
   ratio_1= 1.001953125
   ratio_2= 0.6949462890625
 
10
   ratio_1= 1.001953125
   ratio_2= 0.7054443359375
 
11
   ratio_1= 1.001953125
   ratio_2= 0.7241617838541666
 
12
   ratio_1= 1.001953125
   ratio_2= 0.6370849609375
 
13
   ratio_1= 1.001953125
   ratio_2= 0.7497151692708334
 
14
   ratio_1= 1.001953125
   ratio_2= 0.775634765625
 
15
   ratio_1= 1.001953125
   ratio_2= 0.7765299479166666
 
16
   ratio_1= 1.001953125
   ratio_2= 0.7183024088541666
 
17
   ratio_1= 1.001953125
   ratio_2= 0.7422688802083334
 
18
   ratio_1= 1.001953